In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# # plt.style.use('ggplot')
# import pandas as pd
# import numpy as np
# from PIL import Image
# pd.set_option('display.max_columns', 500)
# import sys, os
# from torch.utils.data import Dataset, DataLoader
# import torch, torchvision
# from torch.autograd import Function
# from torch.autograd import Variable
# from torchvision import transforms
# from torchvision import models
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, auc
# from itertools import cycle
# from itertools import chain
# # from sklearn.linear_model import LogisticRegression
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
# from sklearn.metrics import classification_report
# from imblearn.metrics import specificity_score
# from dalib.modules import WarmStartGradientReverseLayer
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
from data import \
    (
        DermDataset, 
        derm_data, 
        clinic_train, 
        clinic_validate, 
        clinic_test,
        dermoscopic_train,
        dermoscopic_validate,
        dermoscopic_test,
        label_train_diag,
        label_validate_diag,
        label_test_diag,
        label_train_crit,
        label_validate_crit,
        label_test_crit
    )   
from torchvision import transforms
import pandas as pd
import torch

2023-08-30 13:21:52.036701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 13:21:52.178650: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-30 13:21:52.680456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-30 13:21:52.680519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
derm_data.dataset_stats()

Number of cases: 1011
Number of cases to train: 413
Number of cases to validate: 203
Number of cases to test: 395


In [4]:
print(len(clinic_train), len(clinic_validate), len(clinic_test))
# print(clinic_train)

413 203 395


In [5]:
train_transforms = transforms.Compose([transforms.Resize([299, 299]),
                                       # transforms.Pad(padding=10, fill=(255, 176, 145)),
                                       transforms.RandomCrop([299, 299], padding=20, padding_mode='edge'),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomRotation([-45, 45]),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5 , 0.5 , 0.5), (0.5 , 0.5 , 0.5))])
test_transforms = transforms.Compose([transforms.Resize([299, 299]),
    transforms.ToTensor(),
    transforms.Normalize((0.5 , 0.5 , 0.5), (0.5 , 0.5 , 0.5))
])
image_transforms = {'train':train_transforms, 'test':test_transforms}

train = list(zip(clinic_train, dermoscopic_train, label_train_diag, label_train_crit))
train_df = pd.DataFrame(train, columns=['c_path','d_path','lab_diag', 'lab_crit'])
train_dataset = DermDataset(train_df, transform=image_transforms['train'])

validate = list(zip(clinic_validate, dermoscopic_validate, label_validate_diag, label_validate_crit))
validate_df = pd.DataFrame(validate, columns=['c_path','d_path','lab_diag', 'lab_crit'])
validate_dataset = DermDataset(validate_df, transform=image_transforms['test'])

test = list(zip(clinic_test, dermoscopic_test, label_test_diag, label_test_crit))
test_df = pd.DataFrame(test, columns=['c_path','d_path','lab_diag', 'lab_crit'])
test_dataset = DermDataset(test_df, transform=image_transforms['test'])

In [6]:
train_df

,c_path,d_path,lab_diag,lab_crit
0,../data/release_v0/images/NEL/NEL025.JPG,../data/release_v0/images/NEL/Nel026.jpg,0,"[0, 0, 0, 0, 0, 0, 1]"
1,../data/release_v0/images/NEL/Nel032.jpg,../data/release_v0/images/NEL/Nel033.jpg,0,"[0, 0, 0, 0, 2, 0, 1]"
2,../data/release_v0/images/NEL/NEL036.JPG,../data/release_v0/images/NEL/Nel037.jpg,0,"[0, 0, 2, 0, 0, 0, 0]"
3,../data/release_v0/images/NEL/Nel084.jpg,../data/release_v0/images/NEL/Nel085.jpg,0,"[0, 0, 2, 0, 2, 1, 0]"
4,../data/release_v0/images/NEL/NEL088.JPG,../data/release_v0/images/NEL/Nel089.jpg,0,"[0, 0, 2, 0, 2, 0, 1]"
...,...,...,...,...
408,../data/release_v0/images/NEL/Nel067.jpg,../data/release_v0/images/NEL/Nel066.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"
409,../data/release_v0/images/NEL/Nel069.jpg,../data/release_v0/images/NEL/Nel068.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"
410,../data/release_v0/images/NEL/Nel070.jpg,../data/release_v0/images/NEL/Nel071.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"
411,../data/release_v0/images/Ggl/Ggl011.jpg,../data/release_v0/images/Ggl/Ggl012.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"


In [7]:
from torchvision import models
from model import \
    (
        CNN,
        Concate,
        Discriminator,
        ReconstructionNet
    )

In [8]:
device = torch.device("cuda:0")# ("cuda:0")

    
resnet50 = models.resnet50(pretrained=True)
resnet501 = models.resnet50(pretrained=True)
cnn_c = CNN(resnet50).to(device)
cnn_d = CNN(resnet501).to(device)
concate_net = Concate().to(device)
discriminator = Discriminator().to(device)# 判别分布

### Attention based reconstruction

In [9]:
reconstruct_net_c = ReconstructionNet(in_feature=2048 * 2, output_size=(299, 299)).to(device)
reconstruct_net_d = ReconstructionNet(in_feature=2048 * 2, output_size=(299, 299)).to(device)

In [10]:
import torch.nn as nn
import torch.optim as optim
from itertools import chain

In [11]:
learning_rate = 1e-5
learning_rate_re = 1e-5
criterion = nn.CrossEntropyLoss()
opt_list = chain(cnn_c.parameters(), cnn_d.parameters(), concate_net.parameters(), reconstruct_net_c.parameters(), reconstruct_net_d.parameters(), discriminator.parameters())
# optimizer = optim.Adam(chain(reconstruct_net_c.parameters(), reconstruct_net_d.parameters(), concate_net.parameters(), cnn_c.parameters(), cnn_d.parameters()), lr=learning_rate , weight_decay=0.0001) #
optimizer = optim.Adam(opt_list, lr=learning_rate, weight_decay=0.0001) # , weight_decay=0.0001
# optimizer_con = optim.Adam(chain(concate_net.parameters(), discriminator.parameters()), lr=learning_rate) # , weight_decay=0.0001
# optimizer_re = optim.Adam(chain(reconstruct_net_c.parameters(), reconstruct_net_d.parameters()), lr=learning_rate_re) # , weight_decay=0.0001
# criterion_recon = nn.MSELoss()
criterion_recon = nn.MSELoss(reduction='none')
criterion_l1 = nn.L1Loss(reduction='none')


In [20]:
from utils import \
    (
        train_func,
        test_func,
        metric,
        get_average_acc,
        get_average_auc,
        get_confusion_matrix,
        get_specificity,
    )
        

In [18]:
trainloader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=4)
record_acc = 0.
record_auc = 0.

from time import time
total_time_train = 0
for epoch in range(5):
        # training
        start_time_epoch_train = time()

        print("Epoch {} begin train...".format(epoch))
        pred_all_train, label_true_train = train_func(trainloader, 
                cnn_c, 
                cnn_d, 
                concate_net, 
                reconstruct_net_c, 
                reconstruct_net_d, 
                optimizer, 
                criterion, 
                device, 
                epoch
            )
        auc_all_train, acc_all_train, con_all_train = metric(pred_all_train, label_true_train, show=False)
        avg_acc = get_average_acc(acc_all_train)# get the average acc
        avg_auc = get_average_auc(auc_all_train)
        con_metric = get_confusion_matrix(pred_all_train, label_true_train) # compute recall and precision
        specificity = get_specificity(pred_all_train, label_true_train)
        # sens, spec, prec = get_confusion_matrix(con_all_test)
        print(avg_acc, avg_auc)
        # if i % 10 == 0 or i == (epochs - 1):
        if (record_acc+record_auc) <= (avg_acc + avg_auc):
            record_acc = avg_acc
            record_auc = avg_auc
            print("Train metics on epoch {}:".format(epoch))
            print(auc_all_train)
            print(acc_all_train)
            print("Current best average ACC: {:.4f}".format(avg_acc))
            print("Current average AUC: {:.4f}".format(avg_auc))
            print()

        end_time_epoch_train = time()
        epoch_time_train = end_time_epoch_train - start_time_epoch_train
        total_time_train += epoch_time_train

Epoch 0 begin train...
Epoch: 0 train loss, Diag loss: 1.3536, PN loss: 1.0948, STR loss: 0.8821, PIG loss: 0.9458, RS loss: 0.5660, DaG loss: 1.0698, BWV loss: 0.5117, VS loss: 0.6803
0.6180379746835443 0.5404092217795586
Train metics on epoch 0:
{'diag': {0: 0.7274076517150396, 1: 0.2728310502283105, 2: 0.3829729911766687, 3: 0.4559154929577465, 4: 0.3437849944008958, 'micro': 0.757263259093094, 'macro': 0.43867034365317836}, 'pn': {0: 0.492731466580839, 1: 0.4362381030973207, 2: 0.5125329345581429, 'micro': 0.5505399775676975, 'macro': 0.48241649800895425}, 'str': {0: 0.5188067444876783, 1: 0.5395622895622896, 2: 0.5360853891284372, 'micro': 0.7716103188591571, 'macro': 0.533791996215409}, 'pig': {0: 0.5662999269996871, 1: 0.557696926032661, 2: 0.5962385430305917, 'micro': 0.7423489825348502, 'macro': 0.575578331880386}, 'rs': {0: 0.6643925050597376, 1: 0.6643925050597376, 'micro': 0.7962506008652459, 'macro': 0.6674258948520413}, 'dag': {0: 0.5520338983050848, 1: 0.5047420914152848

In [ ]:
fa

In [22]:
record_acc = 0.
record_auc = 0.

model_name_c = './checkpoint/feature_extraction_c_fusion_9-12_21.pth'# './checkpoint/feature_extraction_concate_discrinimator_0713_c1_two_stream.pth' # 3 ahieved the best performance 
model_name_d = './checkpoint/feature_extraction_d_fusion_9-12_21.pth' #'./checkpoint/feature_extraction_concate_discrinimator_0713_d1_two_stream.pth'
model_name_concate = './checkpoint/concatenate_fusion_9-12_21.pth'# './checkpoint/concate_discrinimator_0713_concatenate1_two_stream.pth'
model_name_dis_c = './checkpoint/discriminator_fusion_9-12_21.pth'# './checkpoint/reconstruct_concate_discrinimator_0713_c1_two_stream.pth'
model_name_recon_c = './checkpoint/reconstruct_c_fusion_9-12_21.pth'# './checkpoint/feature_extraction_concate_recon_0713_c1_two_stream.pth' # 3 ahieved the best performance
model_name_recon_d = './checkpoint/reconstruct_c_fusion_9-12_21.pth'# './checkpoint/feature_extraction_concate_recon_0713_d1_two_stream.pth'

checkpoint_c = torch.load(model_name_c)
cnn_c.load_state_dict(checkpoint_c)
checkpoint_d = torch.load(model_name_d)
cnn_d.load_state_dict(checkpoint_d)
checkpoint_concate_net = torch.load(model_name_concate)
concate_net.load_state_dict(checkpoint_concate_net)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=48,
                                              shuffle=False, num_workers=4)
i=1
print("Epoch {} begin testing...".format(i))
pred_all_test, label_true_test = test_func(testloader, 
                        cnn_c, 
                        cnn_d, 
                        concate_net, 
                        reconstruct_net_c, 
                        reconstruct_net_d, 
                        criterion,
                        device,
                        i)
auc_all_test, acc_all_test, con_all_test = metric(pred_all_test, label_true_test, show=False)
avg_acc = get_average_acc(acc_all_test)# get the average acc
avg_auc = get_average_auc(auc_all_test)
con_metric = get_confusion_matrix(pred_all_test, label_true_test) # compute recall and precision
specificity = get_specificity(pred_all_test, label_true_test)
# sens, spec, prec = get_confusion_matrix(con_all_test)
print(avg_acc, avg_auc)
# if i % 10 == 0 or i == (epochs - 1):
if (record_acc+record_auc) <= (avg_acc + avg_auc):
    record_acc = avg_acc
    record_auc = avg_auc
    print("Test metics on epoch {}:".format(i))
    print(auc_all_test)
    print(acc_all_test)
    print("Current best average ACC: {:.4f}".format(avg_acc))
    print("Current average AUC: {:.4f}".format(avg_auc))
    # print('confusion_matrix' + str(con_metric))


Epoch 1 begin testing...
Epoch: 1 test loss, Diag loss: 0.8220, PN loss: 0.6805, STR loss: 0.5907, PIG loss: 0.7436, RS loss: 0.3653, DaG loss: 0.8746, BWV loss: 0.2866, VS loss: 0.4405
0.7591772151898735 0.8473625607985668
Test metics on epoch 1:
{'diag': {0: 0.941787598944591, 1: 0.8965338314653383, 2: 0.8903482184953189, 3: 0.9059859154929577, 4: 0.8171892497200448, 'micro': 0.9339560967793622, 'macro': 0.8917050790802882}, 'pn': {0: 0.8919912026606587, 1: 0.8449139021840788, 2: 0.8205155593534144, 'micro': 0.8611632751161672, 'macro': 0.8542629397745594}, 'str': {0: 0.8300625951615632, 1: 0.8954286454286454, 2: 0.8072736269173676, 'micro': 0.8978464989585003, 'macro': 0.8462718635554005}, 'pig': {0: 0.8073573886745229, 1: 0.8515249759846302, 2: 0.8338590643970956, 'micro': 0.8701778561128024, 'macro': 0.8326071577107976}, 'rs': {0: 0.8676633805575505, 1: 0.8676633805575504, 'micro': 0.9067841692036532, 'macro': 0.8704126520852186}, 'dag': {0: 0.7717627118644068, 1: 0.77678516796181